**REPLICATION CODE: MOURIFIÉ/HENRY/MEANGO**

Start date: 10/11/2025
Author: Jordi Torres Vallverdú

Goal: idea is to replicate the analysis done by Mourifié et al (2020) in another context with another data. The idea is to seek new applications of their methodology.

Notes on the context: I am using data on Belgium region of Flandes in the 1990s. I have a panel of students observed from the beginning of high-school up to two years into higher education. Given that I don't observe future labor outcomes, I will only consider potential outcomes in terms of success probability of having completed the first year of higher education (this is a proxy for completion)


$D=\{1,0\}$, where 1 is entered stem high school track and 0 if entered non-stem high-school track

$Y=\{1,0\}$, where 1 is graduated from stem degree and 0 is graduated from non-stem degree

Z, set of instruments


In [1]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year1/papers_final_memoire/data/clean/roymodel_database.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/set_identification/"


main = DataFrame(load(data_path))

Row,ID,hs_course1,hs_stem1,postsec_type2,postsec_stem2,c_par_beliefpar_HS_E,c_par_beliefpar_postHS,c_par_childage_homechange,c_par_notsureifB,c_par_yearsinkindergarten,c_par_repeatelementaryeduc,c_par_specialeducelem,c_par_income,c_par_home,c_par_spokenlanguage_dutch_fa,c_par_dialect_fa,c_par_spokenlanguage_dutch_mo,c_par_dialect_mo,c_par_parentschoose,c_par_childchoose,c_par_schoolchoose,c_par_otherchoose,c_par_nochoose,c_bao_advies_school_p0,c_bao_advies_progr_p0,c_bao_advies_welkprogr_p0,c_bao_belief_lkr_endpoint_p0,c_bao_special_p0,c_bao_result_dutch_p0,c_bao_result_math_p0,c_bao_result_all_p0,c_iq_male,c_iq_birth_year,c_iq_birth_month,c_iq_WB1,c_iq_NB1,c_iq_NIQ,c_iq_VIQ,c_iq_SIQ,c_iq_IQ,c_iq_opl,c_sskills_CON1,c_sskills_CON2,c_sskills_AGREE1,c_sskills_AGREE2,c_sskills_EXTRA1,c_sskills_CON3,c_sskills_AGREE3,c_sskills_EXTRA2,c_sskills_EXTRA3,c_sskills_workparents,c_sskills_income,c_sskills_IQ_discrete,z_numeric_iq,income_reduced
,Int32?,Float32?,Float32?,Float32?,Float32?,Int32?,Int32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int32?,Int32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int32?,Int32?,Float32?,Int32?,Int32?,Int32?,Int8?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Float32?,Int8?,Float32?
1,11135,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1,7,0.0,5,4,4,1,1978,8,-0.418461,0.609975,91.4869,120.331,109.205,110.171,2,5,5,4,3,4,5,2,4,3,1.0,3.0,5.0,1,2.0
2,11137,6.0,0.0,1.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,1,1,0.0,5,4,5,0,1978,10,-0.332319,0.560348,97.4658,107.186,100.036,102.72,1,5,5,5,5,5,5,5,5,5,1.0,4.0,4.0,2,3.0
3,11154,6.0,1.0,2.0,0.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,5,0.57565,1.49024,106.991,115.378,104.474,111.563,2,5,5,5,5,5,5,5,5,5,missing,1.0,5.0,3,1.0
4,11161,6.0,1.0,1.0,0.0,6,4,missing,0,4,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,missing,0,1978,2,0.221887,1.49837,110.288,115.582,99.4507,111.189,3,5,5,5,5,5,5,5,5,5,1.0,4.0,5.0,4,3.0
5,11162,6.0,1.0,1.0,0.0,6,5,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,5,0,1978,2,0.988978,0.905644,118.363,100.334,90.7207,103.559,1,5,5,5,5,5,5,5,5,4,1.0,3.0,4.0,6,2.0
6,11163,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,6,0.959115,0.984085,120.458,118.775,126.786,125.161,3,5,5,5,5,5,5,5,5,2,1.0,4.0,6.0,6,3.0
7,11166,6.0,1.0,2.0,0.0,6,missing,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,6,missing,0,1978,11,0.158003,1.39593,95.3408,91.9245,89.5506,90.9654,3,5,5,5,5,5,5,5,5,5,missing,1.0,3.0,1,1.0
8,11171,6.0,0.0,1.0,0.0,6,3,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,5,0.83656,1.99263,110.01,125.206,121.763,123.113,2,5,5,5,5,5,5,5,5,5,1.0,3.0,6.0,4,2.0
9,11172,6.0,1.0,2.0,1.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,10,1.0285,0.811891,108.808,96.97,99.3044,101.327,3,5,5,5,5,5,5,5,5,4,missing,1.0,4.0,4,1.0


**1. Data manipulation**

First we create a subsample of observations on gender and socio-economic status, just like they do in their exercise. I will create bins based on this: 

- Gender
- Income categories: 6 brackets
- Minority status (proxied by family background)


In [ ]:
using DataFramesMeta, Statistics 

groups = groupby(main, [:c_sskills_income, :c_iq_male])
subsamples = [DataFrame(g) for g in groups]



Row,ID,hs_course1,hs_stem1,postsec_type2,postsec_stem2,c_par_beliefpar_HS_E,c_par_beliefpar_postHS,c_par_childage_homechange,c_par_notsureifB,c_par_yearsinkindergarten,c_par_repeatelementaryeduc,c_par_specialeducelem,c_par_income,c_par_home,c_par_spokenlanguage_dutch_fa,c_par_dialect_fa,c_par_spokenlanguage_dutch_mo,c_par_dialect_mo,c_par_parentschoose,c_par_childchoose,c_par_schoolchoose,c_par_otherchoose,c_par_nochoose,c_bao_advies_school_p0,c_bao_advies_progr_p0,c_bao_advies_welkprogr_p0,c_bao_belief_lkr_endpoint_p0,c_bao_special_p0,c_bao_result_dutch_p0,c_bao_result_math_p0,c_bao_result_all_p0,c_iq_male,c_iq_birth_year,c_iq_birth_month,c_iq_WB1,c_iq_NB1,c_iq_NIQ,c_iq_VIQ,c_iq_SIQ,c_iq_IQ,c_iq_opl,c_sskills_CON1,c_sskills_CON2,c_sskills_AGREE1,c_sskills_AGREE2,c_sskills_EXTRA1,c_sskills_CON3,c_sskills_AGREE3,c_sskills_EXTRA2,c_sskills_EXTRA3,c_sskills_workparents,c_sskills_income,c_sskills_IQ_discrete,z_numeric_iq,income_reduced
,Int32?,Float32?,Float32?,Float32?,Float32?,Int32?,Int32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int32?,Int32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int32?,Int32?,Float32?,Int32?,Int32?,Int32?,Int8?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Float32?,Int8?,Float32?
1,11166,6.0,1.0,2.0,0.0,6,missing,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,6,missing,0,1978,11,0.158003,1.39593,95.3408,91.9245,89.5506,90.9654,3,5,5,5,5,5,5,5,5,5,missing,1.0,3.0,1,1.0
2,11246,6.0,0.0,1.0,0.0,6,5,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,6,0,1978,11,1.09782,2.19707,114.479,122.349,114.374,120.853,3,5,5,5,5,5,5,5,5,5,1.0,1.0,6.0,5,1.0
3,11272,6.0,1.0,2.0,0.0,6,5,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,missing,missing,missing,0,1978,2,0.476583,1.31293,95.3703,117.185,106.693,109.138,3,5,5,5,5,5,5,5,5,5,1.0,1.0,4.0,1,1.0
4,11311,6.0,1.0,2.0,0.0,6,missing,12,0,3,0,0,2,3,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,1,1,6,0.0,4,5,4,0,1978,10,1.59291,2.14332,112.499,113.799,134.868,122.516,2,5,4,5,5,2,5,4,4,2,1.0,1.0,6.0,5,1.0
5,11457,6.0,0.0,2.0,0.0,6,1,missing,0,3,0,0,missing,1,1.0,0.0,0.0,missing,0.0,0.0,0.0,1.0,0.0,1,1,2,2,0.0,2,3,3,0,1978,12,-0.279718,-0.991823,82.2108,93.7312,92.0622,88.1096,1,2,4,4,2,5,4,5,5,5,1.0,1.0,2.0,1,1.0
6,11473,6.0,0.0,2.0,0.0,6,2,missing,missing,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,2,5,0.0,5,4,4,0,1978,3,-0.460866,-0.650889,88.4975,103.941,94.2812,96.1163,1,4,3,5,5,5,4,4,3,4,1.0,1.0,3.0,1,1.0
7,12652,6.0,0.0,2.0,0.0,6,3,missing,0,4,0,0,missing,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1,1,1,6,0.0,4,4,5,0,1978,9,0.532142,1.07975,104.617,107.015,99.3044,104.887,1,5,5,4,4,4,3,3,5,4,1.0,1.0,4.0,3,1.0
8,12666,4.0,0.0,2.0,0.0,missing,missing,missing,missing,4,0,0,missing,1,1.0,0.0,1.0,0.0,missing,missing,missing,missing,missing,1,1,1,5,0.0,4,4,4,0,1978,3,0.388227,0.443466,100.799,100.566,99.769,100.486,3,3,3,4,4,4,3,4,4,4,missing,1.0,4.0,2,1.0
9,12667,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,6,0.0,5,5,5,0,1978,2,1.04905,2.24525,110.96,126.756,105.768,119.048,3,5,4,5,4,2,4,4,5,4,1.0,1.0,5.0,5,1.0


**2. Test for monothonicity of E(Y/Z=z)**
This boils down to testing whether the probability of Y=1 is increasing in Z. Usual tests include Hu, Shi (2019), Chetverikhov (2013). These tests, however, are done under much more complicated cases as in mine: Hu et al, for example, account for a setting with Y being continuous and Z also being continuous; in my case, I just need to test over Z=1,2,3...

Given that the idea is to use set identification techniques, I will provide some evidence that this holds or not for a given subset of the data. Then I will focus mostly on the el(z) measure. 

In [3]:
size_subsample=10
size_p=3

m= zeros(Float64, size_subsample, size_p, size_p)

10×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [4]:
##Estimate probabilities 

function cell_probs(df, instrument::Symbol)
    zvals = sort(unique(df[!, instrument]))
    k = length(zvals)
    p_k = zeros(Float64, k)
    n_k = zeros(Int, k)

    for (i, z) in enumerate(zvals)
        num = sum((df.postsec_stem2 .== 1) .&
                  (df[!, instrument] .== z))

        n_k[i] = sum(df[!, instrument] .== z)
        p_k[i] = num / n_k[i]
    end

    return p_k, n_k
end



probs= [cell_probs(i, :z_numeric_iq) for i in subsamples]

10-element Vector{Tuple{Vector{Float64}, Vector{Int64}}}:
 ([0.26865671641791045, 0.3269230769230769, 0.2708333333333333, 0.41304347826086957, 0.3076923076923077, 0.4, 0.5714285714285714], [67, 52, 48, 46, 39, 25, 28])
 ([0.5714285714285714, 0.4583333333333333, 0.46875, 0.5, 0.6097560975609756, 0.5952380952380952, 0.6451612903225806], [28, 24, 32, 38, 41, 42, 62])
 ([0.16326530612244897, 0.225, 0.2571428571428571, 0.37142857142857144, 0.17857142857142858, 0.43478260869565216, 0.5], [49, 40, 35, 35, 28, 23, 20])
 ([0.6666666666666666, 0.631578947368421, 0.5555555555555556, 0.6, 0.6666666666666666, 0.625, 0.45], [15, 19, 18, 10, 15, 24, 20])
 ([0.2857142857142857, 0.25, 0.3111111111111111, 0.46808510638297873, 0.3055555555555556, 0.3157894736842105, 0.38095238095238093], [49, 48, 45, 47, 36, 38, 21])
 ([0.3333333333333333, 0.5263157894736842, 0.6551724137931034, 0.6, 0.6071428571428571, 0.6875, 0.65625], [27, 19, 29, 15, 28, 32, 32])
 ([0.1875, 0.3235294117647059, 0.25, 0.225806451612903

In [5]:
function compute_var(p_k, n_k)
    k = length(p_k)
    var = zeros(k)
    for j in 1:k
        var[j] = p_k[j] * (1 - p_k[j]) / n_k[j]
    end
    return var
end


compute_var (generic function with 1 method)

In [6]:
#Generate moment inequalities 
function monotonicity_AS(df, instrument::Symbol)

    p_k, n_k = cell_probs(df, instrument)
    var = compute_var(p_k, n_k)

    K = length(p_k)
    if K < 2
        error("Need at least 2 Z categories for monotonicity test.")
    end

    J = K - 1  # number of inequalities
    g = zeros(J)
    sig = zeros(J)

    for j in 1:J
        g[j] = p_k[j+1] - p_k[j]
        sig[j] = sqrt(var[j+1] + var[j])
    end

    n = nrow(df)
    tvals = zeros(J)
    for j in 1:J
        if sig[j] > 0
            tvals[j] = sqrt(n) * (-min(g[j], 0)) / sig[j]
        else
            tvals[j] = 0.0
        end
    end

    T = maximum(tvals)

    return g, sig, T
end



monotonicity_AS (generic function with 1 method)

In [7]:
T_vals = Float64[]
g_list = []
sig_list = []

for df_sub in subsamples
    g, sig, T = monotonicity_AS(df_sub, :z_numeric_iq)
    push!(T_vals, T)
    push!(g_list, g)
    push!(sig_list, sig)
end

T_vals ##same rejections as before


10-element Vector{Float64}:
 17.75991639216843
 13.37485865109482
 26.801390148796656
 12.937066058166597
 25.89070209784949
  4.825661239408463
 10.804590844148679
  2.6392300227004672
  7.688892903951603
 24.315006171778244

In [8]:
using Distributions
using Random

function boot_t_1(subsamples, instrument::Symbol, b_iter::Int64)

    nsubs = length(subsamples)
    T_dist = zeros(nsubs, b_iter)

    for b in 1:b_iter
        for i in 1:nsubs

            dt = subsamples[i]
            n_i = nrow(dt)

            hat_p = mean(dt.postsec_stem2)

            Y_b = rand.(Bernoulli(hat_p), n_i)

            dt_boot = deepcopy(dt)
            dt_boot.postsec_stem2 = Y_b

            _, _, T_dist[i, b] = monotonicity_AS(dt_boot, instrument)

        end
    end

    return T_dist
end


boot_t_1 (generic function with 1 method)

In [9]:
Random.seed!(12345)

T_dist = boot_t_1(subsamples, :z_numeric_iq, 1000)

# critical values
cv_95 = [quantile(T_dist[i,:], 0.95) for i in 1:length(subsamples)]
cv_90 = [quantile(T_dist[i,:], 0.90) for i in 1:length(subsamples)]
cv_85 = [quantile(T_dist[i,:], 0.85) for i in 1:length(subsamples)]


cv_85

10-element Vector{Float64}:
 34.63496960148269
 32.743631162685205
 31.361168476565915
 22.75106651943767
 35.147491092684206
 27.856953075226386
 31.115998256749503
 27.60078000526138
 32.272652170541065
 26.395254141161246

In [10]:
using DataFrames

function monot_table(subsamples, T_vals, cv_95, cv_90, cv_85)

    n = length(subsamples)

    df_out = DataFrame(
        subsample = Int[],
        income = String[],
        gender = String[],
        T_stat = Float64[],
        cv_95 = Float64[],
        cv_90 = Float64[],
        cv_85 = Float64[],
        reject_95 = Int[],
        reject_90 = Int[],
        reject_85 = Int[]
    )

    for i in 1:n

        income_i = string(unique(subsamples[i].c_sskills_income)[1])
        gender_i = string(unique(subsamples[i].c_iq_male)[1])

        T_i = T_vals[i]

        r95 = T_i >= cv_95[i] ? 1 : 0
        r90 = T_i >= cv_90[i] ? 1 : 0
        r85 = T_i >= cv_85[i] ? 1 : 0

        push!(df_out, (
            i,
            income_i,
            gender_i,
            T_i,
            cv_95[i],
            cv_90[i],
            cv_85[i],
            r95,
            r90,
            r85
        ))
    end

    return df_out
end

df = monot_table(subsamples, T_vals, cv_95, cv_90, cv_85)


Row,subsample,income,gender,T_stat,cv_95,cv_90,cv_85,reject_95,reject_90,reject_85
,Int64,String,String,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,1,1.0,0,17.7599,41.9466,37.3502,34.635,0,0,0
2,2,1.0,1,13.3749,40.1793,35.6684,32.7436,0,0,0
3,3,2.0,0,26.8014,38.5178,33.8102,31.3612,0,0,0
4,4,2.0,1,12.9371,29.796,25.3835,22.7511,0,0,0
5,5,3.0,0,25.8907,42.1774,38.5596,35.1475,0,0,0
6,6,3.0,1,4.82566,34.6674,30.5691,27.857,0,0,0
7,7,4.0,0,10.8046,39.1669,34.6403,31.116,0,0,0
8,8,4.0,1,2.63923,34.7965,30.8509,27.6008,0,0,0
9,9,5.0,0,7.68889,39.3692,35.3513,32.2727,0,0,0


**USING PAV**

In [11]:
#Estimate test

function pav3(pk, nk)
    p1, p2, p3 = pk
    n1, n2, n3 = nk

  #monothonicity holds
  if p1 ≤ p2 ≤ p3
    return pk
  end


  #case 1
  if p1>=p2
    
    θ_12= (p1*n1 + p2*n2)/(n1+n2)

    if p3>=θ_12
      return[θ_12, θ_12, p3]
    else 
    θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)

    return[θ_123,θ_123,θ_123]
    end 
  end 
  
  #case 2
  if p2>=p3

    θ_23= (p2*n2 + p3*n3)/(n2+n3)

    if p1<θ_23
      return[p1, θ_23, θ_23]
    else 
      θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)
     return[θ_123,θ_123,θ_123]
    end 

  end 


end


pav3 (generic function with 1 method)

In [12]:
##extend the function to apply to any number of pair comparisons. 

In [13]:
#To check it works: 

pk = [0.40, 0.20, 0.50]
nk = [50, 60, 40]

pav3(pk, nk)


3-element Vector{Float64}:
 0.2909090909090909
 0.2909090909090909
 0.5

In [14]:
p_hat= zeros(length(subsamples), length(zvals))

for i in 1:length(subsamples)

    pk, nk = probs[i]

    p_hat[i,:]= pav3(pk,nk)
end 

p_hat

UndefVarError: UndefVarError: `zvals` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
n=zeros(length(subsamples))
T=zeros(length(subsamples))
[n[i]= nrow(subsamples[i]) for i in 1:length(subsamples)]

for i in 1:length(subsamples)

    pk, nk= probs[i]
    p_hat[i,:]= pav3(pk,nk)

    T[i]= sqrt(n[i])* maximum(pk .- p_hat[i,:])
end

T

UndefVarError: UndefVarError: `p_hat` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
for i in 1:length(subsamples)
    println((i, unique(subsamples[i].c_sskills_income), unique(subsamples[i].c_iq_male), T[i]))
end


(1, Union{Missing, Float32}[1.0], Union{Missing, Int8}[0], 0.0)
(2, Union{Missing, Float32}[1.0], Union{Missing, Int8}[1], 0.0)
(3, Union{Missing, Float32}[2.0], Union{Missing, Int8}[0], 0.0)
(4, Union{Missing, Float32}[2.0], Union{Missing, Int8}[1], 0.0)
(5, Union{Missing, Float32}[3.0], Union{Missing, Int8}[0], 0.0)
(6, Union{Missing, Float32}[3.0], Union{Missing, Int8}[1], 0.0)
(7, Union{Missing, Float32}[4.0], Union{Missing, Int8}[0], 0.0)
(8, Union{Missing, Float32}[4.0], Union{Missing, Int8}[1], 0.0)
(9, Union{Missing, Float32}[5.0], Union{Missing, Int8}[0], 0.0)
(10, Union{Missing, Float32}[5.0], Union{Missing, Int8}[1], 0.0)


In [17]:
using Distributions
using Random
Random.seed!(12345)   #seed



T_dist= zeros(length(subsamples), b_iter)

##set seed!!!
function boot_t(instrument::Symbol, b_iter::Int64)

 T_dist= zeros(length(subsamples), b_iter)

  for b in 1:b_iter
    for i in 1:length(subsamples)

          dt = subsamples[i]   ##then we loop over i 
          p_iso = p_hat[i,:]   
          Z = dt[!, instrument] 
          p_obs = p_iso[Z]   
          Y_b = rand.(Bernoulli.(p_obs))

          dt_boot = deepcopy(dt)       
          dt_boot.postsec_stem2 = Y_b   #simul replace

          pk_b, nk_b= cell_probs(dt_boot)
          p_iso_b= pav3(pk_b,nk_b)
          T_dist[i,b]= sqrt(n[i])* maximum(pk_b .- p_iso_b)
    end 
  end
  
  return T_dist

end 


T_dist= boot_t(:z_numeric_iq, 1000)





UndefVarError: UndefVarError: `b_iter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
using Statistics

c_value_95= zeros(length(subsamples))
c_value_90= zeros(length(subsamples))

[c_value_95[i]= quantile(T_dist[i, :], 0.95) for i in 1:length(subsamples)]
[c_value_90[i]= quantile(T_dist[i, :], 0.90) for i in 1:length(subsamples)]



10-element Vector{Float64}:
 37.350213910057185
 35.668361064885275
 33.8102060281684
 25.38350663499
 38.55956633912844
 30.569140888106478
 34.64029592313845
 30.85085939543906
 35.35125534496015
 29.04426318905199

In [19]:
t_result95= zeros(length(subsamples))
t_result90= zeros(length(subsamples))

for i in 1:length(subsamples)
    if (T[i]>=c_value_95[i])
        t_result95[i]=1
    else 
        t_result95[i]=0
    end

    if (T[i]>=c_value_90[i])
        t_result90[i]=1
    else 
        t_result90[i]=0
    end


end 

tests= [t_result95 t_result90]

10×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

**Using IQ as instrument**-->classic Manski Pepper type of instrument






**2. Estimating bounds on the L(z) measure**

The goal is now to understand what explains the violations of Roy selection model using the same method as in Mourifié, Henry, Meango (2020). Will keep the same example going, but, if needed, I will also include a continuous instrument: example can be numeric IQ estimated at baseline. 

The main issue with this literature is that sample analogs of $\Theta_I$ are the suprema and infima of estimated bounding functions and they have substantial finite sample bias and estimated bounds tend to be much tighter than population bounds. -->Manski/Pepper (2001).

I think I need to treat the instrument as continuous; if not it is meaningless. Because CRL seems to be devised with this property. Then I probably need to adapt the test for monothonicity too. But so far, I have something, which is fine.

Start simple; support is discrete; then we do the same exercise with a continuous instrument. 

In [20]:
##Estimate probabilities 

function cell_probs_0(df, instrument::Symbol)

    zvals = sort(unique(df[!, instrument]))  
    k = length(zvals)

    p_k = Dict{Int, Float64}()   
    n_k = Dict{Int, Int}()

    for z in zvals
        sub = df[df[!, instrument] .== z, :]

        n = nrow(sub)
        n_k[z] = n

        if n == 0
            p_k[z] = NaN
        else
            p_k[z] = sum(sub.postsec_stem2 .== 0) / n
        end
    end

    return zvals, p_k, n_k
end


probs= [cell_probs_0(i, :c_iq_opl) for i in subsamples]



10-element Vector{Tuple{Vector{Union{Missing, Int8}}, Dict{Int64, Float64}, Dict{Int64, Int64}}}:
 ([1, 2, 3], Dict(2 => 0.6355140186915887, 3 => 0.5666666666666667, 1 => 0.75), Dict(2 => 107, 3 => 90, 1 => 108))
 ([1, 2, 3], Dict(2 => 0.3924050632911392, 3 => 0.45794392523364486, 1 => 0.4444444444444444), Dict(2 => 79, 3 => 107, 1 => 81))
 ([1, 2, 3], Dict(2 => 0.7246376811594203, 3 => 0.6052631578947368, 1 => 0.7560975609756098), Dict(2 => 69, 3 => 38, 1 => 123))
 ([1, 2, 3], Dict(2 => 0.4473684210526316, 3 => 0.3157894736842105, 1 => 0.40625), Dict(2 => 38, 3 => 19, 1 => 64))
 ([1, 2, 3], Dict(2 => 0.626984126984127, 3 => 0.6904761904761905, 1 => 0.7297297297297297), Dict(2 => 126, 3 => 84, 1 => 74))
 ([1, 2, 3], Dict(2 => 0.4507042253521127, 3 => 0.35185185185185186, 1 => 0.42105263157894735), Dict(2 => 71, 3 => 54, 1 => 57))
 ([1, 2, 3], Dict(2 => 0.6515151515151515, 3 => 0.7074829931972789, 1 => 0.7777777777777778), Dict(2 => 66, 3 => 147, 1 => 18))
 ([1, 2, 3], Dict(2 => 0.45652

In [21]:
function compute_el(df, instrument::Symbol)

    zvals, p_k, n_k = cell_probs_0(df, instrument)

    el = zeros(length(zvals))

    running_min = Inf

    for (i, z) in enumerate(zvals)
        running_min = min(running_min, p_k[z])
        el[i] = p_k[z] - running_min
    end

    return zvals, p_k, n_k, el
end


zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)


(Union{Missing, Int8}[1, 2, 3], Dict(2 => 0.6153846153846154, 3 => 0.45323741007194246, 1 => 0.25), Dict(2 => 13, 3 => 139, 1 => 4), [0.0, 0.3653846153846154, 0.20323741007194246])

In [22]:
#Generate moment inequalities

function m_ineq(theta, p_k, n_k, zvals_s)
    n = length(zvals_s)
    moment = zeros(n,n)
    se_moment = zeros(n,n)

    for ii in 1:n
        z_i = zvals_s[ii]

        for jj in 1:ii
            z_j = zvals_s[jj]

            if n_k[z_i] == 0 || n_k[z_j] == 0
                moment[ii,jj] = 0.0
                se_moment[ii,jj] = Inf
                continue
            end

            moment[ii,jj] = (p_k[z_i] - p_k[z_j]) - theta[ii] ##key point
            var_i = p_k[z_i]*(1 - p_k[z_i]) / n_k[z_i]
            var_j = p_k[z_j]*(1 - p_k[z_j]) / n_k[z_j]

            se_moment[ii,jj] = sqrt(var_i + var_j)
        end
    end

    return moment, se_moment
end



m_ineq (generic function with 1 method)

In [23]:
#put the moment inequalities in a vector

function vectorize(moment::AbstractMatrix)
 n = size(moment, 1)

 K = n * (n+ 1) ÷ 2
 moment_vector= zeros(K)
    k=1
        for j in 1:n
            for y in 1:j
                    moment_vector[k]=moment[j,y]
                    k +=1
            end 
            
        end
     
  return moment_vector

end 

vectorize (generic function with 1 method)

In [24]:
##Now generate T value of the moment for a given value of the grid. 

function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [25]:

function bootstrap(df, moment_not,se_moment_not,theta_not, B, instrument::Symbol, zvals_original)

    T_b= zeros(B)
    for b in 1:B

        df_b = nothing
        while true
            n = nrow(df)
            df_try = df[rand(1:n, n), :]
            zvals_b = sort(unique(df_try[!, instrument]))
            if zvals_b == zvals_original
                df_b = df_try
                break
            end
        end

        zvals, p_k_b, n_k_b, el_b= compute_el(df_b, instrument)
        
        moment, se_moment=m_ineq(theta_not, p_k_b, n_k_b, zvals)

        moment_v_b= vectorize(moment)

        Z_vec_b= @. (moment_v_b - moment_not) /se_moment_not
        T_b[b]=maximum(Z_vec_b)

    end 
    return T_b

end 

bootstrap (generic function with 1 method)

In [26]:
#let's take only the case where it is violated, the main condition...



zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)

theta_not=el 
moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

se_not_vec= vectorize(se_moment_not)
moment_not_vec=vectorize(moment_not)


zvals_original= sort(unique(subsamples[10][!, :c_iq_opl]))



3-element Vector{Union{Missing, Int8}}:
 1
 2
 3

In [27]:
T_boot= bootstrap(subsamples[10], moment_not_vec,se_not_vec,theta_not, 1000, :c_iq_opl, zvals_original)


K_alpha= quantile(T_boot, 0.95)



1.9276600632010683

In [28]:
##Apply the code for each subsample and for different instruments. 


function apply_bootsubsamples(instrument::Symbol, B)

    size_s= length(subsamples)
    T_boot_s=zeros(size_s, B)
    
    for s in 1:size_s
        
        zvals, p_k, n_k, el = compute_el(subsamples[s], instrument)

        theta_not=el #should I change this?
        moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

        se_not_vec= vectorize(se_moment_not)
        moment_not_vec=vectorize(moment_not)


        zvals_original= sort(unique(subsamples[s][!, instrument]))

        T_boot_s[s, :]= bootstrap(subsamples[s], moment_not_vec,se_not_vec,theta_not, B, instrument, zvals_original)

        

    end 

    return T_boot_s

end

apply_bootsubsamples (generic function with 1 method)

In [29]:
T_boot_s= apply_bootsubsamples(:c_iq_opl, 1000)


iz= length(subsamples)

alpha_by_subsample= zeros(iz)

[alpha_by_subsample[i]= quantile(T_boot_s[i,:], 0.95) for i in 1:iz]





10-element Vector{Float64}:
 2.06492922987482
 2.088657846915591
 2.0613780829562622
 2.117723317757581
 2.0566944141878354
 2.143904623625604
 2.065272794281873
 2.2524722086929656
 2.355818789785681
 1.9629147790763186

In [30]:
##Finally set up the grid, compute T and then compare with critical values simulated via bootstrap. 

#set a grid of values for el(z) do I need to loop over these or jus tby computing the upper bound is enough... ? is it really needed more?

In [31]:
function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [32]:
T_test(subsamples[10], :c_iq_opl, theta_not)

theta_not

3-element Vector{Float64}:
 0.0
 0.3653846153846154
 0.20323741007194246

In [33]:
#compute te

function T_simul(subsamples, θ_grid, instrument::Symbol)

    iz = length(subsamples)              
    grid_size = length(θ_grid)           

    T_grid = zeros(iz, grid_size)

    for z in 1:iz
        df_s = subsamples[z]
        for theta_idx in 1:grid_size
            θ = θ_grid[theta_idx]
            T_grid[z, theta_idx] = T_test(df_s, instrument, θ)
        end
    end

    return T_grid
end


T_simul (generic function with 1 method)

In [34]:

grid1 = [0.0]                     # quite ad hoc coded; needs to in
grid2 = collect(0.0:0.005:0.35)   
grid3 = collect(0.0:0.005:0.185)   


θ_grid = [ [x,y,z] for (x,y,z) in Iterators.product(grid1, grid2, grid3) ]

T_grid= T_simul(subsamples, θ_grid, :c_iq_opl)






10×2698 Matrix{Float64}:
 0.0       0.0       0.0        0.0        …  0.0        0.0        0.0
 0.897041  0.897041  0.897041   0.897041      0.0        0.0        0.0
 0.0       0.0       0.0        0.0           0.0        0.0        0.0
 0.405645  0.356319  0.306992   0.257666      0.0        0.0        0.0
 0.957133  0.957133  0.957133   0.957133      0.0        0.0        0.0
 0.336526  0.279779  0.223033   0.166286   …  0.0        0.0        0.0
 0.803827  0.803827  0.803827   0.803827      0.0        0.0        0.0
 0.139094  0.114212  0.0893292  0.0644469     0.0        0.0        0.0
 0.0       0.0       0.0        0.0           0.0        0.0        0.0
 1.43226   1.41266   1.39306    1.37346       0.0995042  0.0826774  0.0826774

In [35]:
function accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)

    ns = size(T_grid, 1)      
    ng = size(T_grid, 2)       

    accepted = Vector{Vector{Vector{Float64}}}(undef, ns)

    for s in 1:ns
        k_s = alpha_by_subsample[s]

        θ_accepted = Vector{Vector{Float64}}()

        for g in 1:ng
            if T_grid[s, g] <= k_s
                push!(θ_accepted, θ_grid[g])
            end
        end

        accepted[s] = θ_accepted
    end

    return accepted
end


accepted_theta_per_subsample (generic function with 1 method)

In [36]:
T_grid = T_simul(subsamples, θ_grid, :c_iq_opl)

accepted = accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)


10-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [

**Extension to account for Z being continuous**--> this should go more in line with Chernozhukov Lee Rosen. 

In [ ]:
##Case where Z is continuous. This is following their comments p.708-709
#CORRECT shit!

In [4]:
using DataFrames, Statistics

function cell_probs_0(df::DataFrame, instrument::Symbol, yvar::Symbol)

    zvals = sort(unique(df[!, instrument]))
    k = length(zvals)
    n = nrow(df)

    beta_hat = Vector{Float64}(undef, k)   
    n_k      = Vector{Int}(undef, k)      

    for (j, z) in enumerate(zvals) #j is position and z is value in a vector
        idx = df[!, instrument] .== z
        nj = count(idx)
        n_k[j] = nj

        if nj == 0
            beta_hat[j] = NaN
        else
            beta_hat[j] = mean(df[idx, yvar] .== 0)
        end
    end

    return zvals, beta_hat, n_k, n, k
end


cell_probs_0 (generic function with 1 method)

In [5]:
zvals, beta_hat, n_k, n, k = cell_probs_0(subsamples[1], :c_iq_opl, :postsec_stem2)


(Union{Missing, Int8}[1, 2, 3], [0.75, 0.6355140186915887, 0.5666666666666667], [108, 107, 90], 305, 3)

In [6]:
#3.Estimate asymptotic variance

using LinearAlgebra

function variance_compute(beta_hat::Vector{Float64}, n_k::Vector{Int}, n::Int)
    k = length(beta_hat)
    Ωdiag = zeros(Float64, k)

    for j in 1:k
         if n_k[j] == 0 || isnan(beta_hat[j])
            Ωdiag[j] = NaN

         else 
        Ωdiag[j] = n * beta_hat[j] * (1.0 - beta_hat[j]) / n_k[j]
         end 
    end
    Ω = Diagonal(Ωdiag)
    sqrt_Ω= Diagonal(sqrt.(Ωdiag))
    return Ω, sqrt_Ω
end


variance_compute (generic function with 1 method)

In [7]:
Ω, Ω_sqrt=variance_compute(beta_hat, n_k, n)

([0.5295138888888888 0.0 0.0; 0.0 0.6602707007019346 0.0; 0.0 0.0 0.8321604938271605], [0.7276770498572075 0.0 0.0; 0.0 0.8125704281488064 0.0; 0.0 0.0 0.912228312335876])

In [8]:
function define_moments_for_j(beta_hat::Vector{Float64}, j::Int)
    k = length(beta_hat)

    V = collect(1:j-1)         
    M = length(V)

    p_list = Vector{Vector{Float64}}(undef, M)
    theta_hat = Vector{Float64}(undef, M)

    for (m, i) in enumerate(V)
        p = zeros(Float64, k)
        p[j] =  1.0
        p[i] = -1.0
        p_list[m] = p

        theta_hat[m] = beta_hat[j] - beta_hat[i]
    end

    return V, p_list, theta_hat
end


define_moments_for_j (generic function with 1 method)

In [9]:
#step 5

using LinearAlgebra

function compute_g_and_s(p_list::Vector{Vector{Float64}}, Ω_sqrt::Diagonal, n::Int)
    M= length(p_list)
    k=length(p_list[1])

    g_hats = Vector{Vector{Float64}}(undef, M)
    s_n    = Vector{Float64}(undef, M)

    for m in 1:M
        g_hats[m]= vec(p_list[m]'*Ω_sqrt)
        s_n[m] = norm(g_hats[m]) / sqrt(n)
    end 

    return g_hats, s_n
end



compute_g_and_s (generic function with 1 method)

In [10]:
#step 6

using LinearAlgebra, Statistics

function clr_critical_value(g_hats::Vector{Vector{Float64}}, R::Int, n::Int; rng=Random.default_rng())

    M = length(g_hats)           
    K = length(g_hats[1])        

    γ_n = 1 - 0.1 / log(n)

    T = zeros(Float64, R)

    for r in 1:R
        Zr = randn(rng, K)

        best = -Inf
        vals = zeros(M)
        for m in 1:M
            g = g_hats[m]
            val = dot(g, Zr) / norm(g)

            if val>best 
                best=val
            end    
        end

        T[r] = best
    end

    k_nV = quantile(T, γ_n)

    return k_nV
end


clr_critical_value (generic function with 1 method)

In [11]:
function restrict_moments(theta_hat::Vector{Float64}, k_nV::Float64, s_n::Vector{Float64})

    M = length(theta_hat)
    B = similar(theta_hat)

    for m in 1:M
        B[m] = theta_hat[m] + k_nV * s_n[m]
    end
    
    b = minimum(B)

    V_hat_idx = Int[]

    for m in 1:M
        if theta_hat[m] <= b + 2 * k_nV * s_n[m]
            push!(V_hat_idx, m)
        end
    end

    return V_hat_idx
end


restrict_moments (generic function with 1 method)

In [12]:
function clr_lower_bound(theta_hat, s_n, k_nVhat)
    minimum(theta_hat .+ k_nVhat .* s_n)
end


clr_lower_bound (generic function with 1 method)

In [13]:
function clr_lower_bound_for_j(theta_hat, s_n, k_nVhat, Vhat)

    return minimum(theta_hat[m] .+ k_nVhat .* s_n[m] for m in Vhat)

end 

clr_lower_bound_for_j (generic function with 1 method)

In [14]:
function clr_el_discrete(df::DataFrame, Z::Symbol, Y::Symbol; R=2000, rng=Random.default_rng())
    zvals, beta_hat, n_k, n, k = cell_probs_0(df, Z, Y)
    _, Ωsqrt = variance_compute(beta_hat, n_k, n)

    el_L = fill(NaN, k)
    el_L[1] = 0.0 #by default, first point in discrete case will be 00

    for j in 2:k
        V, p_list, theta_hat = define_moments_for_j(beta_hat, j)
        g_hats, s_n = compute_g_and_s(p_list, Ωsqrt, n)

        k_nV = clr_critical_value(g_hats, R, n)
        Vhat = restrict_moments(theta_hat, k_nV, s_n)

        el_L[j] = minimum(theta_hat[m] + k_nV*s_n[m] for m in Vhat)
    end

    return zvals, beta_hat, el_L
end


clr_el_discrete (generic function with 1 method)

In [18]:

using Random
Random.seed!(12345)

N= length(subsamples)





results= [clr_el_discrete(i, :z_numeric_iq, :postsec_stem2; R=2000, rng=Random.default_rng()) for i in subsamples]




10-element Vector{Tuple{Vector{Union{Missing, Int8}}, Vector{Float64}, Vector{Float64}}}:
 ([1, 2, 3, 4, 5, 6, 7], [0.7313432835820896, 0.6730769230769231, 0.7291666666666666, 0.5869565217391305, 0.6923076923076923, 0.6, 0.42857142857142855], [0.0, 0.11609115300459427, 0.19565244397594214, 0.08058687341961981, 0.19324512236273264, 0.15563853369149316, -0.01767739101505955])
 ([1, 2, 3, 4, 5, 6, 7], [0.42857142857142855, 0.5416666666666666, 0.53125, 0.5, 0.3902439024390244, 0.40476190476190477, 0.3548387096774194], [0.0, 0.3895793680508444, 0.29305869675706114, 0.2598287127161814, 0.15763444721015224, 0.17244067937119628, 0.12108802717776113])
 ([1, 2, 3, 4, 5, 6, 7], [0.8367346938775511, 0.775, 0.7428571428571429, 0.6285714285714286, 0.8214285714285714, 0.5652173913043478, 0.5], [0.0, 0.12363251327759953, 0.1263450437548857, 0.037152185188470355, 0.2111575060012647, 0.03191393148421584, -0.028398814883390355])
 ([1, 2, 3, 4, 5, 6, 7], [0.3333333333333333, 0.3684210526315789, 0.44444444